In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [121]:
version = 'v2'

# R  W  11
# R  P  11
# G  W  11
# G  P  11
# H  W  11
# H  P  11

# category = 'R'
# category = 'G'
category = 'H'

# bet_type = 'W'
bet_type = 'P'

In [122]:
engine = create_engine('sqlite:///../../data/race.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

9069 races loaded!


In [123]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

2290 H races!


In [124]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head(3)

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
36,37,CHARLTON,VIC,M,H,2017-10-01,1,DNR LOGISTICS 3YO PACE MS,2017-10-01 02:14:00,Paying,2100,"[[1], [5], [3], [4]]",6,"[{""runnerName"": ""TORQUE TOTHE STARZ"", ""runnerN...",3.3,15.6,36.1,57.3,"[[1], [5], [3], [4]]","[{'runnerName': 'TORQUE TOTHE STARZ', 'runnerN..."
37,38,CHARLTON,VIC,M,H,2017-10-01,2,AGISTMENT KILMORE PACE MS,2017-10-01 02:48:00,Paying,1609,"[[1], [2], [8], [7]]",8,"[{""runnerName"": ""SUBTLE EMBRACE"", ""runnerNumbe...",4.3,10.5,33.8,186.1,"[[1], [2], [8], [7]]","[{'runnerName': 'SUBTLE EMBRACE', 'runnerNumbe..."
38,39,CHARLTON,VIC,M,H,2017-10-01,3,VALE ROD GRANT TROT MS,2017-10-01 03:18:00,Paying,2100,"[[5], [1], [6], [2]]",10,"[{""runnerName"": ""SHESASUNDON"", ""runnerNumber"":...",6.5,14.0,91.7,249.9,"[[5], [1], [6], [2]]","[{'runnerName': 'SHESASUNDON', 'runnerNumber':..."


In [125]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
    data_all = data_all.append(race['runners'])    
data_all.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
3,0.118624,0.050412,0.013706,0.011834,0,0,0,"{'returnWin': 19, 'returnWinOpen': 18, 'return...",True,6,...,5.0,0.044420,CLAY CRAIB,MM'S SKITTLE,2,B MATTINGLY,32.7,0.030581,6.0,0.025635
4,0.631735,0.268468,0.377579,0.326016,0,0,1,"{'returnWin': 2.4, 'returnWinOpen': 3.1, 'retu...",True,6,...,3.0,0.219762,TRACE TETRICK,REAL FUTURE,3,R REDDER,2.2,0.454545,1.0,0.381029
5,0.457279,0.194330,0.154411,0.133324,0,0,3,"{'returnWin': 2.9, 'returnWinOpen': 2.9, 'retu...",True,6,...,4.0,0.189794,ROSS LEONARD,SKYWAY BILLY,4,J LLOYD,4.7,0.212766,3.0,0.178354


In [126]:
# drop scratched
data = data_all.dropna(subset=['win_odds', 'place_odds'])
data = data[(data.win_odds > 0)]
data = data[(data.place_odds > 0)]
data.describe()

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,num_runners,place_odds,place_perc,place_rank,place_scaled,runnerNumber,win_odds,win_perc,win_rank,win_scaled
count,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000,21766.0,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000
mean,0.315200,0.105210,0.095747,0.105210,0.836948,0.0,1.004916,10.224111,5.288170,0.348781,5.527061,0.105210,5.823532,24.899535,0.124666,5.590003,0.105210
std,0.205309,0.072407,0.135580,0.141096,3.700307,0.0,1.399164,2.909468,5.561978,0.241800,3.361828,0.076834,3.498805,29.787295,0.143416,3.367806,0.120703
min,0.005955,0.001936,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.006667,1.000000,0.001886,1.000000,1.040000,0.001600,1.000000,0.001349
25%,0.139017,0.045835,0.014941,0.016905,0.000000,0.0,0.000000,8.000000,2.000000,0.153846,3.000000,0.045536,3.000000,6.200000,0.030960,3.000000,0.026148
50%,0.264656,0.086650,0.041458,0.048551,0.000000,0.0,0.000000,10.000000,3.500000,0.285714,5.000000,0.085064,5.000000,14.300000,0.069930,5.000000,0.059235
75%,0.471019,0.154669,0.116952,0.133578,0.000000,0.0,2.000000,12.000000,6.500000,0.500000,8.000000,0.147182,8.000000,32.300000,0.161290,8.000000,0.136333
max,0.999987,1.000000,0.999997,1.000000,80.000000,0.0,4.000000,20.000000,150.000000,1.000000,20.000000,1.000000,20.000000,625.000000,0.961538,20.000000,1.000000


In [127]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    21766.000000
mean         0.296242
std          0.456609
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
dtype: float64

In [128]:
xn = data['num_runners']
xwp = data['win_perc']
xws = data['win_scaled']
xwr = data['win_rank']
xpp = data['place_perc']
xps = data['place_scaled']
xpr = data['place_rank']
X = pd.concat([xn, xwp, xws, xwr, xpp, xps, xpr], axis=1)
X.describe()

,num_runners,win_perc,win_scaled,win_rank,place_perc,place_scaled,place_rank
count,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000,21766.000000
mean,10.224111,0.124666,0.105210,5.590003,0.348781,0.105210,5.527061
std,2.909468,0.143416,0.120703,3.367806,0.241800,0.076834,3.361828
min,1.000000,0.001600,0.001349,1.000000,0.006667,0.001886,1.000000
25%,8.000000,0.030960,0.026148,3.000000,0.153846,0.045536,3.000000
50%,10.000000,0.069930,0.059235,5.000000,0.285714,0.085064,5.000000
75%,12.000000,0.161290,0.136333,8.000000,0.500000,0.147182,8.000000
max,20.000000,0.961538,1.000000,20.000000,1.000000,1.000000,20.000000


In [129]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 500
print('epochs = {}'.format(epochs))

layer_1 = 50
layer_2 = 50

tag = '{}{}x{}{}'.format(category, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/each_way/{}/models/{}.h5'.format(version, tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 7
epochs = 500
tag = H50x50P
model loaded


In [130]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/each_way/{}/summary/{}'.format(version, tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file
model.save(file_name)
print('model saved')


model saved
